# T1202 Indirect Command Execution

-----------------------------------------------------------------------

## Technique Description

Adversaries may abuse utilities that allow for command execution to bypass security restrictions that limit the use of command-line interpreters. Various Windows utilities may be used to execute commands, possibly without invoking [cmd](https://attack.mitre.org/software/S0106). For example, [Forfiles](https://attack.mitre.org/software/S0193), the Program Compatibility Assistant (pcalua.exe), components of the Windows Subsystem for Linux (WSL), as well as other utilities may invoke the execution of programs and commands from a [Command and Scripting Interpreter](https://attack.mitre.org/techniques/T1059), Run window, or via scripts. (Citation: VectorSec ForFiles Aug 2017) (Citation: Evi1cg Forfiles Nov 2017)

Adversaries may abuse these features for [Defense Evasion](https://attack.mitre.org/tactics/TA0005), specifically to perform arbitrary execution while subverting detections and/or mitigation controls (such as Group Policy) that limit/prevent the usage of [cmd](https://attack.mitre.org/software/S0106) or file extensions more commonly associated with malicious payloads.

## Technique Detection

Monitor and analyze logs from host-based detection mechanisms, such as Sysmon, for events such as process creations that include or are resulting from parameters associated with invoking programs/commands/files and/or spawning child processes/network connections. (Citation: RSA Forfiles Aug 2017)

-----------------------------------------------------------------------

### Tactics:

  *   Defense-Evasion

### Platforms:

  * Windows

### Defenses Bypassed:

  * Static File Analysis

  * Application Control

### Data Sources:

  * **Process:** Process Creation

  * **Command:** Command Execution

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| Lazarus Group | [Lazarus Group](https://attack.mitre.org/groups/G0032) persistence mechanisms have used <code>forfiles.exe</code> to execute .htm files.(Citation: Qualys LolZarus)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1202)

  * [Evi1Cg Forfiles Nov 2017](https://twitter.com/Evi1cg/status/935027922397573120), Evi1cg. (2017, November 26). block cmd.exe ? try this :. Retrieved January 22, 2018.

  * [Rsa Forfiles Aug 2017](https://community.rsa.com/community/products/netwitness/blog/2017/08/14/are-you-looking-out-for-forfilesexe-if-you-are-watching-for-cmdexe), Partington, E. (2017, August 14). Are you looking out for forfiles.exe (if you are watching for cmd.exe). Retrieved January 22, 2018.

  * [Vectorsec Forfiles Aug 2017](https://twitter.com/vector_sec/status/896049052642533376), vector_sec. (2017, August 11). Defenders watching launches of cmd? What about forfiles?. Retrieved January 22, 2018.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Information Here

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- Attackers are constantly finding new methods to achieve code execution outside of the "known" LOLbins avoiding signatures and simplistic detections. Astaroth malware is a good example of new techniques being used such as: ExtExport.exe (Utility shipped with Default Internet Explorer).
- This DLL Hijack abuse allows another method for a valid "Microsoft" binary to load a malicious payload for execution without the use of "common" methods such as "rundll". 
- Understanding that there will always be another "unintended" method for execution, but identifying other malicious artifacts/behaviors will help identify these.

## Analytical References

  * [Upgrades Delivery Support Infrastructure - Revenge RAT Malware Bigger Threat (cofense)](https://cofense.com/upgrades-delivery-support-infrastructure-revenge-rat-malware-bigger-threat/)
  * [Abusing COM Objects (github)](https://0xpat.github.io/Abusing_COM_Objects/)
  * [Exploit-DB](https://www.exploit-db.com/docs/48767)
  * [Indirect Command Execution 2020 (pentestlab)](https://pentestlab.blog/2020/07/06/indirect-command-execution/)
  * [cmdHijack - Command Argument Confusion with Path Traversal in cmd.exe (hackingiscool)](https://hackingiscool.pl/cmdhijack-command-argument-confusion-with-path-traversal-in-cmd-exe/)
  * [The COM Library (microsoft)](https://docs.microsoft.com/en-us/windows/win32/com/the-com-library)
  * [Detecting Adversary Tradecraft with Image Load Event Logging and EQL (medium)](https://medium.com/threatpunter/detecting-adversary-tradecraft-with-image-load-event-logging-and-eql-8de93338c16)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** Windows API (C, C++, C#) programs

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** Having access to the underlying API avoids calls to cmd.exe/powershell.exe, but will still generate a CreateProcess event.
      - Identifying parent processes calling net.exe, systeminfo, whoami, ipconfig.exe and other telemetry based commands can help identify behavior of initial reconnaissance. 

  * **Query:** ```event_id: (1 OR 4688)```

#### Analytic 2

  * **Information:** Windows Utilities

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** forfiles - has been observed in the wild for bypassing command interpreter logging
      - pcalua - program compatibility assitant that runs when it detects a software with compatibility issues
      - conhost - provides an interface between command prompt and Windows explore
      - explorer - 
      - waitfor - used to synchronize computers across a network by sending signals
      - SyncAppvPublishingServer.vbs - initiates the Microsoft application virtualization (App-V) publishing refresh operation.

  * **Query Pseudo:** ```event_id: 1 and process_name (waitfor | conhost | pcalua | forfiles) and unique command_line```

#### Analytic 3

  * **Information:** Windows COM Interface

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** Research has shown ability to use COM interfaces for command execution and obscuring parent/child process creation.
  
      - Based off of Microsoft's documentation for a program to use a COM interface they have to initialize and uninitialize the COM library. Instead of looking for specific DLLs loaded for each COM surface its better to look for the minimum required to call a COM any interface. (primarily Ole32.dll and Rpcss.exe)
          - MMC20.Application
          - WScript.Shell
          - ShellBrowserWindow
          - Shell.Application

  * **Query Pseudo:** Identify Processes utilizing COM ```event_id: 7 and ImageName: (*ole* | *rpcss*)``` followed by a CreateProcess event `event_id: (1 | 4688)`


-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

